# Databases Final Data Import

The purpose of this notebook is to import final project data into MongoDB.

In [14]:
import pandas as pd
from pymongo import MongoClient
from dotenv import dotenv_values
import numpy as np
import random
from bson import ObjectId

In [3]:
df = pd.read_csv('../../data/ratings_and_sentiments.csv', encoding='ISO-8859-1')

df.head()

,coffee_shop_name,review_text,rating,num_rating,cat_rating,bool_HIGH,overall_sent,vibe_sent,tea_sent,service_sent,seating_sent,price_sent,parking_sent,location_sent,alcohol_sent,coffee_sent,food_sent,hours_sent,internet_sent,local_sent
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the vib...,5.0 star rating,5.0,HIGH,1.0,4.0,3,0.0,0.0,0.0,0.0,0,0.0,1.0,3,0,0.0,0.0,0.0
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, vibe ...",4.0 star rating,4.0,HIGH,1.0,3.0,3,0.0,0.0,0.0,0.0,0,0.0,0.0,0,2,0.0,0.0,0.0
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in food seating ...,4.0 star rating,4.0,HIGH,1.0,2.0,2,0.0,0.0,3.0,0.0,0,0.0,0.0,-1,2,0.0,0.0,0.0
3,The Factory - Cafe With a Soul,11/25/2016 Very cool vibe! Good drinks Nice s...,2.0 star rating,2.0,LOW,0.0,1.0,0,0.0,0.0,-1.0,-1.0,0,0.0,0.0,0,0,0.0,0.0,0.0
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are location within...,4.0 star rating,4.0,HIGH,1.0,2.0,0,0.0,0.0,0.0,0.0,3,0.0,0.0,0,0,0.0,0.0,0.0


In [4]:
df.isna().sum()

coffee_shop_name       5
review_text            5
rating                 5
num_rating             5
cat_rating             5
bool_HIGH              5
overall_sent           5
vibe_sent           3114
tea_sent               5
service_sent           5
seating_sent           5
price_sent             5
parking_sent           5
location_sent          5
alcohol_sent           5
coffee_sent            5
food_sent              5
hours_sent             6
internet_sent          5
local_sent             5
dtype: int64

In [5]:
# show the number of na values in each column
df.isna().sum()

# drop rows with no coffee_shop_name
df = df.dropna(subset=['coffee_shop_name'])

# drop rows with no review_text
df = df.dropna(subset=['review_text'])

# drop rows with no rating
df = df.dropna(subset=['num_rating'])

In [6]:
df.isna().sum()

coffee_shop_name       0
review_text            0
rating                 0
num_rating             0
cat_rating             0
bool_HIGH              0
overall_sent           0
vibe_sent           3109
tea_sent               0
service_sent           0
seating_sent           0
price_sent             0
parking_sent           0
location_sent          0
alcohol_sent           0
coffee_sent            0
food_sent              0
hours_sent             1
internet_sent          0
local_sent             0
dtype: int64

In [7]:
# list the number of unique values in each column
df.nunique()

coffee_shop_name      78
review_text         6915
rating                 5
num_rating             5
cat_rating             2
bool_HIGH              2
overall_sent           9
vibe_sent              9
tea_sent               8
service_sent           9
seating_sent           8
price_sent             7
parking_sent           9
location_sent          9
alcohol_sent           7
coffee_sent            9
food_sent              9
hours_sent             7
internet_sent          7
local_sent             6
dtype: int64

In [8]:
reviews = df[['coffee_shop_name', 'review_text', 'num_rating']]

# remove trailing whitespace from the coffee shop names and reviews
reviews['coffee_shop_name'] = reviews['coffee_shop_name'].str.strip()
reviews['review_text'] = reviews['review_text'].str.strip()

reviews['num_rating'] = reviews['num_rating'].astype(float)

reviews.head()

C:\Users\jankowskitb\AppData\Local\Temp\ipykernel_7024\766521197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['coffee_shop_name'] = reviews['coffee_shop_name'].str.strip()
C:\Users\jankowskitb\AppData\Local\Temp\ipykernel_7024\766521197.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['review_text'] = reviews['review_text'].str.strip()
C:\Users\jankowskitb\AppData\Local\Temp\ipykernel_7024\766521197.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,coffee_shop_name,review_text,num_rating
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the vibe...,5.0
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, vibe i...",4.0
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in food seating I...,4.0
3,The Factory - Cafe With a Soul,11/25/2016 Very cool vibe! Good drinks Nice se...,2.0
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are location within ...,4.0


In [9]:
records = reviews.to_dict(orient='records')

# show the first 5 records
for record in records[:5]:
    print(record)

{'coffee_shop_name': 'The Factory - Cafe With a Soul', 'review_text': '11/25/2016 1 check-in Love love loved the vibe! Every corner of the coffee shop had its own style, and there were swings!!! I ordered the matcha coffee, and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad, which included all beverage selections that ranged from coffee to alcohol, desired level of sweetness, and a checkout system. I got my coffee within minutes!  I was hoping for a typical heart or feather on my coffee, but found myself listing out all the possibilities of what the vibe may be. Any ideas?', 'num_rating': 5.0}
{'coffee_shop_name': 'The Factory - Cafe With a Soul', 'review_text': '12/2/2016 Listed in Date Night: Austin, vibe in Austin BEAUTIFUL!!!! Love the vibe! Instagram-worthy!!!  Definitely $$$, so be prepared. This is gonna cost you a pretty penny :) food food was just decent...nothing to rave about.  But, will probably be back just to be somewher

In [2]:
config = dotenv_values('.env')

# Connect to MongoDB
client = MongoClient(config['ATLAS_URI'])
db = client[config['DB_NAME']]

In [10]:
collection = db['reviews']

# Drop the collection if it exists
collection.drop()

# Create the collection
collection = db.create_collection('reviews')

result = collection.insert_many(records)

print(f"Inserted {len(result.inserted_ids)} records into the database.")

Inserted 7616 records into the database.


In [11]:
collection.create_index("coffee_shop_id")

'coffee_shop_id_1'

In [15]:
# Add coffee_shop_id to reviews
coffee_shops = {doc['name']: ObjectId(doc['_id']) for doc in db['coffee_shops'].find({}, {'_id': 1, 'name': 1})}

# Update all review documents to include the coffee_shop_id
for review in collection.find():
    shop_name = review['coffee_shop_name']
    if shop_name in coffee_shops:
        collection.update_one(
            {'_id': review['_id']},
            {'$set': {'coffee_shop_id': coffee_shops[shop_name]}}
        )

print("Reviews updated with coffee_shop_id field")

Reviews updated with coffee_shop_id field


In [29]:
# Create a new DataFrame for coffee shops with required schema fields
shops_data = []

# Get all unique coffee shop names
unique_shops = df['coffee_shop_name'].unique()

# Dictionary of Milwaukee neighborhoods to use for mock locations
milwaukee_areas = [
    {"area": "Downtown", "address": "310 W Wisconsin Ave"},
    {"area": "Third Ward", "address": "400 N Water St"},
    {"area": "Bay View", "address": "2301 S Kinnickinnic Ave"},
    {"area": "East Side", "address": "1800 N Farwell Ave"},
    {"area": "Riverwest", "address": "701 E Center St"},
    {"area": "Walker's Point", "address": "605 S 5th St"},
    {"area": "Washington Heights", "address": "5900 W North Ave"},
    {"area": "Historic Mitchell Street", "address": "1101 W Mitchell St"},
    {"area": "Harbor District", "address": "600 E Greenfield Ave"},
    {"area": "Marquette", "address": "1442 W Wells St"}
]

# Common coffee shop features to extract from sentiment data
feature_mapping = {
    'wifi': {'col': 'internet_sent', 'threshold': 0, 'name': 'Free WiFi'},
    'parking': {'col': 'parking_sent', 'threshold': 0, 'name': 'Parking Available'},
    'food': {'col': 'food_sent', 'threshold': 0, 'name': 'Food Options'},
    'vibe': {'col': 'vibe_sent', 'threshold': 1, 'name': 'Great Atmosphere'},
    'service': {'col': 'service_sent', 'threshold': 0, 'name': 'Friendly Service'},
    'seating': {'col': 'seating_sent', 'threshold': 0, 'name': 'Comfortable Seating'},
    'price': {'col': 'price_sent', 'threshold': 0, 'name': 'Reasonable Prices'},
    'alcohol': {'col': 'alcohol_sent', 'threshold': 0, 'name': 'Serves Alcohol'},
    'tea': {'col': 'tea_sent', 'threshold': 0, 'name': 'Tea Selection'},
    'hours': {'col': 'hours_sent', 'threshold': 0, 'name': 'Extended Hours'},
    'local': {'col': 'local_sent', 'threshold': 0, 'name': 'Locally Owned'},
    'coffee': {'col': 'coffee_sent', 'threshold': 1, 'name': 'Quality Coffee'}
}

# For each coffee shop, create a document with name, location, and features
for shop_name in unique_shops:
    # Get all reviews for this shop
    shop_reviews = df[df['coffee_shop_name'] == shop_name]
    
    # Randomly select a location for this coffee shop
    location = random.choice(milwaukee_areas)
    
    # Extract features based on sentiment analysis in reviews
    features = []
    
    for feature, config in feature_mapping.items():
        # Calculate average sentiment for this feature
        if config['col'] in shop_reviews.columns:
            # Ensure the column is numeric before calculating the mean
            avg_sentiment = pd.to_numeric(shop_reviews[config['col']], errors='coerce').mean()
            # If the average sentiment is above the threshold, add this feature
            if avg_sentiment > config['threshold'] and not np.isnan(avg_sentiment):
                features.append(config['name'])
    
    # Add some randomness to features to make the data more interesting
    if random.random() < 0.3:
        features.append("Outdoor Seating")
    if random.random() < 0.2:
        features.append("Study Friendly")
    if random.random() < 0.15:
        features.append("Pet Friendly")
    if random.random() < 0.1:
        features.append("Live Music")
    
    # Create the document
    shop_doc = {
        'name': shop_name.strip(),
        'location': {
            'area': location['area'],
            'address': location['address'],
            'city': 'Milwaukee',
            'state': 'WI',
            'coordinates': {
                'lat': round(43.04 + random.uniform(-0.05, 0.05), 6),  # Milwaukee area coordinates
                'lng': round(-87.91 + random.uniform(-0.05, 0.05), 6)
            }
        },
        'features': features,
        'rating': round(shop_reviews['num_rating'].mean(), 1) if 'num_rating' in shop_reviews else None,
        'review_count': len(shop_reviews)
    }
    
    shops_data.append(shop_doc)

# Create or update the coffee_shops collection
coffee_shops_collection = db['coffee_shops']
coffee_shops_collection.drop()  # Drop if exists
result = coffee_shops_collection.insert_many(shops_data)

print(f"Inserted {len(result.inserted_ids)} coffee shops into the database.")

# Display a sample coffee shop document
print("\nSample coffee shop document:")
print(coffee_shops_collection.find_one())

Inserted 78 coffee shops into the database.

Sample coffee shop document:
{'_id': ObjectId('67fc7a1f8000455dfec4ddaa'), 'name': 'The Factory - Cafe With a Soul', 'location': {'area': 'Downtown', 'address': '310 W Wisconsin Ave', 'city': 'Milwaukee', 'state': 'WI', 'coordinates': {'lat': 43.042394, 'lng': -87.922401}}, 'features': ['Parking Available', 'Food Options', 'Friendly Service', 'Comfortable Seating', 'Serves Alcohol', 'Tea Selection', 'Extended Hours'], 'rating': 4.4, 'review_count': 244}


In [ ]:
collection = db['coffee_shops']

# print the datatypes of a result document
sample_doc = collection.find_one()
print("\nSample coffee shop document datatypes:")
for key, value in sample_doc.items():
    print(f"{key}: {type(value)}")


Sample coffee shop document datatypes:
_id: <class 'bson.objectid.ObjectId'>
name: <class 'str'>
location: <class 'dict'>
features: <class 'list'>
rating: <class 'float'>
review_count: <class 'int'>


In [ ]:
# 